In [49]:
import numpy as np
import pandas as pd
import json
import os
import shutil
import sys
import gym
import ray

In [50]:
sys.path.insert(0,"")

from Manhattan_Graph_Environment.graphs.ManhattanGraph import ManhattanGraph
from gym_graphenv.envs.GraphworldManhattan import GraphEnv

In [51]:
env=GraphEnv()

get_available_actions() Execution time: 0.30615711212158203 seconds


In [52]:
hubs=env.hubs

In [53]:
import pandas as pd
trips=pd.read_csv("/Users/agnieszkalenart/iCloud Drive (Archive)/Documents/mannheim/team_projekt/ines-autonomous-dispatching-1/data/trips/preprocessed_trips.csv")
trips

,Unnamed: 0,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_node,dropoff_node,pickup_distance,dropoff_distance,route,route_timestamps
0,0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,1825841704,42449985,23.488315,73.871139,"[1825841704, 1825841743, 1825841693, 434755007...","{1825841704: '2016-03-14 17:24:55', 1825841743..."
1,1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,42436590,1692433941,1.403374,15.878506,"[42436590, 42442889, 42440330, 1692450029, 424...","{42436590: '2016-06-12 00:43:35', 42442889: '2..."
2,2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,42430378,42448593,116.850732,41.139491,"[42430378, 42430384, 42432818, 42432825, 59677...","{42430378: '2016-01-19 11:35:24', 42430384: '2..."
3,3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,42436056,42428368,18.344932,85.864692,"[42436056, 4207143418, 246858445, 246858448, 2...","{42436056: '2016-04-06 19:32:31', 4207143418: ..."
4,4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,42428709,42427915,27.389466,114.921239,"[42428709, 42428705, 42428701, 42428695, 42428...","{42428709: '2016-03-26 13:30:55', 42428705: '2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1452665,1458639,id2376096,2,2016-04-08 13:31:04,2016-04-08 13:44:02,4,-73.982201,40.745522,-73.994911,40.740170,N,778,42449308,42429659,28.302655,35.574543,"[42449308, 42446934, 42446933, 8482948064, 424...","{42449308: '2016-04-08 13:31:04', 42446934: '2..."
1452666,1458640,id1049543,1,2016-01-10 07:35:15,2016-01-10 07:46:10,1,-74.000946,40.747379,-73.970184,40.796547,N,655,42427477,1061531730,23.395490,8.407268,"[42427477, 42427427, 42427426, 42427423, 42428...","{42427477: '2016-01-10 07:35:15', 42427427: '2..."
1452667,1458641,id2304944,2,2016-04-22 06:57:41,2016-04-22 07:10:25,1,-73.959129,40.768799,-74.004433,40.707371,N,764,42436489,42440270,64.528027,30.482430,"[42436489, 42436492, 42428007, 42428010, 59679...","{42436489: '2016-04-22 06:57:41', 42436492: '2..."
1452668,1458642,id2714485,1,2016-01-05 15:56:26,2016-01-05 16:02:39,1,-73.982079,40.749062,-73.974632,40.757107,N,373,42445908,596775870,4.854552,47.644393,"[42445908, 42445909, 42445910, 42445651, 42445...","{42445908: '2016-01-05 15:56:26', 42445909: '2..."


In [32]:
hubs

[42447132,
 42446977,
 42430333,
 42433066,
 42442475,
 1919595922,
 42430253,
 596775930,
 42423774,
 3099327976,
 42445867,
 4145735059,
 42445916,
 42445018,
 205024444,
 42428980,
 42427968,
 561042199,
 9177424868,
 42449029,
 42436486,
 5779545445,
 371239958,
 42438809,
 1692433919,
 42424025,
 3099327970,
 42446959,
 248708582,
 595295501,
 42435684,
 3579432156,
 42447105,
 42443674,
 42445357,
 42443534,
 42429342,
 561035358,
 370924957,
 42450468,
 371188750,
 42427769,
 42437644,
 42442937,
 1061531596,
 42427863,
 595352904,
 42450025,
 42440743,
 42428863,
 595314119,
 370888100,
 42440798,
 42432856,
 42430361,
 42442415,
 42430375,
 42433058,
 42457401,
 42450820,
 42444457,
 42433422,
 7490266268,
 42432889,
 42428174,
 42453934,
 1825841704,
 42432985,
 370913758,
 42445392]

In [64]:
hub_1=['42442475']
hub_2=['42430253']

In [65]:
#hubs_string = [str(x) for x in hubs]
#mask = trips.route.apply(lambda x: any(item for item in hubs_string if item in x))
mask1 = trips.route.apply(lambda x: any(item for item in hub_1 if item in x))
selected_trips = trips[mask1]
mask2 = selected_trips.route.apply(lambda x: any(item for item in hub_2 if item in x))
selected_trips2 = selected_trips[mask2]
selected_trips2

,Unnamed: 0,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_node,dropoff_node,pickup_distance,dropoff_distance,route,route_timestamps
1090110,1094632,id2323350,2,2016-06-14 22:11:55,2016-06-14 22:32:04,6,-73.9972,40.736706,-73.975372,40.78706,N,1209,42429374,42442475,12.599793,13.250962,"[42429374, 42430253, 42440710, 42438784, 42428...","{42429374: '2016-06-14 22:11:55', 42430253: '2..."


In [66]:
selected_trips2.iloc[0,17]

"{42429374: '2016-06-14 22:11:55', 42430253: '2016-06-14 22:12:05', 42440710: '2016-06-14 22:12:40', 42438784: '2016-06-14 22:13:01', 42428493: '2016-06-14 22:13:12', 42440721: '2016-06-14 22:13:21', 42434352: '2016-06-14 22:13:33', 42431453: '2016-06-14 22:13:43', 42434354: '2016-06-14 22:13:52', 42434355: '2016-06-14 22:14:00', 42434357: '2016-06-14 22:14:08', 42437283: '2016-06-14 22:14:26', 42449893: '2016-06-14 22:14:34', 42439170: '2016-06-14 22:14:42', 42439178: '2016-06-14 22:15:06', 42439181: '2016-06-14 22:15:32', 5286408252: '2016-06-14 22:15:56', 42438362: '2016-06-14 22:16:22', 42439191: '2016-06-14 22:16:32', 5137743341: '2016-06-14 22:16:51', 5137747688: '2016-06-14 22:17:06', 589100267: '2016-06-14 22:17:14', 42422509: '2016-06-14 22:17:44', 42457925: '2016-06-14 22:17:55', 370913779: '2016-06-14 22:18:04', 370914027: '2016-06-14 22:18:41', 370915061: '2016-06-14 22:19:07', 370924957: '2016-06-14 22:20:09', 371188750: '2016-06-14 22:21:03', 371188756: '2016-06-14 22:21: